Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [2]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [3]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [4]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [5]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---

---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [25]:
batch_size = 512
layers = 5 # number layers, including inputs
layer_nodes = [image_size*image_size, 1024, 1024, 512, 512]
actv_func = [tf.nn.relu, tf.nn.tanh, tf.nn.sigmoid, tf.nn.relu, tf.nn.relu]
l2_lmbd = [0.00001, 0.00001, 0.00001, 0.00001, 0.00001]

def nn_transform(tf_train_dataset, dropout = False):
    logits = tf_train_dataset
    for i in range(layers):
        logits = tf.matmul(logits, weights[i]) + biases[i]
        if i != layers-1: 
            logits = tf.nn.dropout(actv_func[i](logits), 0.5) if dropout else actv_func[i](logits)
    return logits
        
# create a graph(tensorflow concept) to compute on
graph = tf.Graph()
with graph.as_default():
    
    # this placeholder acts as a layer abstraction for the nn
    tf_train_dataset = tf.placeholder(tf.float32, shape = (batch_size, layer_nodes[0]))    
    tf_train_labels = tf.placeholder(tf.float32, shape = (batch_size, num_labels))
    
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # next we create the connecting params between layers
    weights = [tf.Variable(tf.truncated_normal([layer_nodes[i], layer_nodes[i + 1]], stddev = 0.03)) for i in range(layers - 1)]\
               + [tf.Variable(tf.truncated_normal([layer_nodes[-1], num_labels], stddev = 0.03))]          
     
    biases = [tf.Variable(tf.zeros([layer_nodes[i]])) for i in range(1,layers)]\
              + [tf.Variable(tf.zeros([num_labels]))]
    
    
    logits = nn_transform(tf_train_dataset, True)
    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))\
      + sum([l2_lmbd[i]*tf.nn.l2_loss(weights[i]) for i in range(layers)])
        
    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, 1000, 0.96)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(nn_transform(tf_valid_dataset))
    test_prediction = tf.nn.softmax(nn_transform(tf_test_dataset))
    
    
num_steps = 16001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.332729
Minibatch accuracy: 9.0%
Validation accuracy: 10.0%
Minibatch loss at step 500: 0.597687
Minibatch accuracy: 82.2%
Validation accuracy: 84.1%
Minibatch loss at step 1000: 0.590366
Minibatch accuracy: 81.2%
Validation accuracy: 85.5%
Minibatch loss at step 1500: 0.424478
Minibatch accuracy: 88.1%
Validation accuracy: 86.5%
Minibatch loss at step 2000: 0.478335
Minibatch accuracy: 85.5%
Validation accuracy: 87.0%
Minibatch loss at step 2500: 0.387091
Minibatch accuracy: 87.7%
Validation accuracy: 87.4%
Minibatch loss at step 3000: 0.379514
Minibatch accuracy: 88.1%
Validation accuracy: 87.9%
Minibatch loss at step 3500: 0.414403
Minibatch accuracy: 88.1%
Validation accuracy: 88.2%
Minibatch loss at step 4000: 0.419599
Minibatch accuracy: 87.3%
Validation accuracy: 88.5%
Minibatch loss at step 4500: 0.407107
Minibatch accuracy: 86.9%
Validation accuracy: 88.8%
Minibatch loss at step 5000: 0.400808
Minibatch accuracy: 87.5%
Validation accuracy

---

---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

---

---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

---

---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---
